# Assigment 4 - Geodesic distance fields and paths
## Edoardo Vassallo - S4965918
### Done on igl 2.2.1

In [224]:
import igl
import numpy as np
import meshplot as mp

from collections import deque

## 1 - Geodesic Graph

In [225]:
# AUXILLARY FUNCTIONS

# Compute the shortest distance from point pt to segment [a, b].
def point_to_segment_distance(pt, a, b) -> float:
    # Vector from a to b
    ab = b - a
    # Project (pt - a) onto ab, parameterized by t
    t = np.dot(pt - a, ab) / np.dot(ab, ab)
    # Clamp t to [0, 1] to stay within segment
    t = np.clip(t, 0.0, 1.0)
    # Closest point on segment
    projection = a + t * ab
    # Euclidean distance
    return np.linalg.norm(pt - projection)

# Compute 2D intersection between two segments p1p2 and w1w2
# if not found, return the closest vertex w to p1p2
def compute_2D_intersection_or_closest(p1, p2, w1, w2,):
    r = p2 - p1
    s = w2 - w1
    cross_r_s = np.cross(r, s)

    # If lines are not parallel
    if abs(cross_r_s) > 1e-8:
        # Compute parameters t and u for the intersection point
        q_minus_p = w1 - p1
        t = np.cross(q_minus_p, s) / cross_r_s
        u = np.cross(q_minus_p, r) / cross_r_s

        # Check if intersection lies on both segments
        if 0.0 <= t <= 1.0 and 0.0 <= u <= 1.0:
            return p1 + t * r

    # Otherwise, pick the closer w-endpoint
    return w1 if point_to_segment_distance(w1, p1, p2) < point_to_segment_distance(w2, p1, p2) else w2

# Compute the 3D intersection point between the dual edge p1-p2 and edge w1-w2.
def compute_3D_intersection(V, idx_p1, idx_p2, idx_w1, idx_w2,):
    # vertex coordinates
    p1_3d = np.asarray(V[idx_p1])
    p2_3d = np.asarray(V[idx_p2])
    w1_3d = np.asarray(V[idx_w1])
    w2_3d = np.asarray(V[idx_w2])

    # Define 2D frame along w1->w2
    origin = w1_3d
    edge_vec = w2_3d - w1_3d
    length = np.linalg.norm(edge_vec)
    x_axis = edge_vec / length

    # Normal of plane (z-axis)
    ref_vec = p1_3d - w1_3d
    normal = np.cross(x_axis, ref_vec)
    normal /= np.linalg.norm(normal)
    # y-axis orthogonal to x and normal
    y_axis = np.cross(normal, x_axis)

    # 2D coords of p1 and w1,w2
    w1_2d = np.array([0.0, 0.0])
    w2_2d = np.array([length, 0.0])
    p1_2d = np.array([np.dot(p1_3d - origin, x_axis), np.dot(p1_3d - origin, y_axis)])

    # 2D coords of p2 via circle intersection
    r1 = np.linalg.norm(p2_3d - origin)
    r2 = np.linalg.norm(p2_3d - w2_3d)
    d = length
    # Distance along x-axis to circle intersection midpoint
    a = (r1**2 - r2**2 + d**2) / (2 * d)
    # Height from midpoint
    h = np.sqrt(max(r1**2 - a**2, 0.0))

    mid = np.array([a, 0.0])
    # two candidate points
    candidates = [mid + np.array([0.0, h]), mid - np.array([0.0, h])]
    # choose the one opposite side of p1
    side_p1 = np.sign(np.cross(w2_2d - w1_2d, p1_2d - w1_2d))
    p2_2d = next(
        c for c in candidates
        if np.sign(np.cross(w2_2d - w1_2d, c - w1_2d)) == -side_p1
    )

    # Compute 2D intersection or closest on w-segment
    intersect_2d = compute_2D_intersection_or_closest(p1_2d, p2_2d, w1_2d, w2_2d)

    # Map back to 3D
    return origin + intersect_2d[0] * x_axis + intersect_2d[1] * y_axis


# given face i, find the vertex vp opposite 
# to the edge (b, c) on the adjacent face
def find_opposite_vertex(i, b, c, F, TT):
    face = F[i]
    for j in range(3):
        # look for edge (b, c) in face i
        v0, v1 = face[j], face[(j+1) % 3]
        if {v0, v1} == {b, c}:
            # get the triangle-triangle adjacency index for this edge
            nbr = TT[i, j]
            if nbr == -1:
                return None  # Boundary edge, no adjacent face
            # Adjacent face F[nbr] contains [vp, b, c]
            # Find the vertex in F[nbr] that is not b or c
            for vp in F[nbr]:
                if vp not in (b, c):
                    return int(vp)
    return None  # Edge not found in face i



In [226]:
# Build graph of geodesic distances on a mesh
def build_geodesic_graph(V, F):
    n = V.shape[0]  # number of vertices
    adj = [[] for _ in range(n)]  # adjacency list: for each vertex, list of (neighbor, weight)

    # Compute triangle-triangle adjacency for dual edges
    TT, TTi = igl.triangle_triangle_adjacency(F)

    # For each face
    for f_idx in range(F.shape[0]):
        tri = F[f_idx] 

        # For each corner of the face
        for loc in range(3):
            v  = tri[loc]            # current vertex index
            w1 = tri[(loc + 1) % 3]  # next vertex index around the triangle
            w2 = tri[(loc + 2) % 3]  # other vertex index

            # Primal edges
            for w in (w1, w2):
                d = np.linalg.norm(V[v] - V[w])
                # add primal edge
                adj[v].append((w, d, None)) # (neighbor, weight, dual edge intersection)
                adj[w].append((v, d, None))

            # Dual edge
            # find vertex vp which composes the dual edge with v
            vp = find_opposite_vertex(f_idx, w1, w2, F, TT,) 
            if vp is None:
                continue

            # Compute vectors for angle calculations at shared edge vertex w1
            v_w1_v  = V[v]  - V[w1]    # vector w1->v
            v_w1_w2 = V[w2] - V[w1]    # vector w1->w2 (edge direction)
            v_w1_vp = V[vp] - V[w1]    # vector w1->vp 

            # norms of these vectors
            n_w1_v  = np.linalg.norm(v_w1_v)
            n_w1_w2 = np.linalg.norm(v_w1_w2)
            n_w1_vp = np.linalg.norm(v_w1_vp)

            # corner angles at w1 in both triangles
            alfa = np.arccos(np.clip(np.dot(v_w1_v, v_w1_w2) / (n_w1_v * n_w1_w2), -1, 1))
            beta = np.arccos(np.clip(np.dot(v_w1_vp, v_w1_w2) / (n_w1_vp * n_w1_w2), -1, 1))

            # Geodesic length between v and vp via w1
            dual_len = np.sqrt(n_w1_v**2 + n_w1_vp**2 - 2 * n_w1_v * n_w1_vp * np.cos(alfa + beta))

            # compute intersection point of the dual edge with the primal edge w1-w2
            intersection = compute_3D_intersection(V, v, vp, w1, w2)
            
            # add dual edge
            adj[v].append((vp, dual_len, intersection))  # (neighbor, weight, intersection point)
            adj[vp].append((v, dual_len, intersection))  
    return adj

## 2 - Geodesic Solver

In [227]:
# Compute geodesic distances from sources
def geodesic_distances(adj, sources):
    n = len(adj)
    prev     = [(None, None)] * n
    dist     = np.full(n, np.inf)
    in_queue = [False] * n
    Q        = deque()

    # Initialize
    for s in sources:
        dist[s]      = 0.0
        Q.append(s)
        in_queue[s] = True

    while Q:
        # LLL check: compare front label to queue average
        avg = sum(dist[x] for x in Q) / len(Q)
        if dist[Q[0]] > avg:
            # If the front label is larger than the average,
            # move it to the back to maintain small-label-first order
            Q.append(Q.popleft())
            continue

        u = Q.popleft()
        in_queue[u] = False
        du = dist[u]

        for v, w, intersection in adj[u]:
            alt = du + w
            # if the new distance is shorter, update it
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = (u, intersection)
                if not in_queue[v]:
                    # SLF: small‑label‑first
                    if Q and alt < dist[Q[0]]:
                        # if smaller than the first element in 
                        # the queue, append to front
                        Q.appendleft(v)
                    else:
                        # otherwise, append to the back
                        Q.append(v)
                    in_queue[v] = True

    return dist, prev


In [228]:
# Load the test mesh
V, F = igl.read_triangle_mesh("./data/bunny_1k.obj")
#mp.plot(V, F, shading={"wireframe": True})

In [229]:
# Compute geodesic distances
adj = build_geodesic_graph(V, F)

src = [0]

dist, prev = geodesic_distances(adj, src)

# apply color map
plotter = mp.plot(V, F, c=dist, shading={"wireframe": False})

# mark the source with a red point
plotter.add_points(V[[src], :], c=np.array([[1.0, 0.0, 0.0]]), shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001504…

1

## 3. Shortest path between two points [Optional]

In [230]:
# extract path from target to a source
def extract_geodesic_path(target, sources, prev, V):
    path = []
    current = target
    while current is not None:
        path.append(V[current])
        if current in sources:
            break  # stop if we reach a source
        current, intersection = prev[current]  # get previous vertex and eventual intersection point
        if intersection is not None:
            # dual edge intersection point
            path.append(intersection)  
    return path

# plot extracted geodesic path on the mesh
def plot_geodesic_path(V, F, dist, path, src, target):
    p = mp.plot(V, F, c=dist, shading={"wireframe": False, "colormap" : "spring"})

    # mark sources
    p.add_points(V[src, :], c=np.array([[1.0, 0.0, 0.0]]), shading={"point_size": 0.02})
    # mark target
    p.add_points(V[[target], :], c=np.array([[0.0, 1.0, 0.0]]), shading={"point_size": 0.02})

    # draw path
    if len(path) >= 2:
        for i in range(len(path) - 1):
            a = path[i]
            b = path[i+1]
            p.add_lines(a, b, shading={"line_width": 0.05})

    return p


In [231]:
#tgt = 146000 # example for bunny_500k.obj
tgt = 300 # example for bunny_1k.obj

# Extract geodesic path from target to source
path = extract_geodesic_path(tgt, src, prev, V)

# Visualize the geodesic path
# a different color map is used in order to better see the traced path
plot_geodesic_path(V, F, dist, path, src, tgt)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001504…